# Yelp Data Mining Project
## CMPE - 255 Data Mining Fall 2017
### Group 6
- Dhrumil Shah
- Nishant Rathi
- Rashmi Sharma

## Notebook to Preprocess Data and create CSR

In this Jupyter Notebook, we are loading data from yelp data set sql dump. We have considered data using following constraints:
- Users who have written 50 or more reviews
- Their reviews
- Rating of each of the review

This data is grouped into 80% Training data and 20% Test data.
We have grouped data by user ID and then split it.

In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
def test_train_split_per_groupby(review_data,groupby, testperc):
    uids=review_data[groupby].unique()
    full_size = len(uids)
    choose = (int)(((float)(testperc)/100) * full_size)
    indices = np.random.choice(len(uids), choose, replace=False)
    full_indices = [m for m in range(0,len(uids))]
    train_indices = set(full_indices)-set(indices)
    grouped =review_data.groupby([groupby])
    test_indices_final=[]  
    train_indices_final = []
    for g in grouped.groups:
        full_size = len(grouped.groups[g])
        choose = (int)(((float)(testperc)/100) * full_size)
        indices = np.random.choice(full_size, choose, replace=False)
        full_indices = [m for m in range(0,full_size)]
        train_indices = set(full_indices)-set(indices)
        train_indices = list(train_indices)
        test_indices_final.extend(list(grouped.groups[g][indices]))
        train_indices_final.extend(list(grouped.groups[g][train_indices]))
    saveGroupedData("data/test_"+str(groupby)+".dat",test_indices_final)
    saveGroupedData("data/train_"+str(groupby)+".dat",train_indices_final)

def saveGroupedData(filename, indices):
    output_file = open(filename, 'w')
    for i in indices:
        output_file.write(str(i)+"\n")
    output_file.close()    

In [3]:
review_data_all = pd.read_csv('data/reviews_no_text.dat',"\t")
train_indices_user = test_train_split_per_groupby(review_data_all,'UID',20)

### Creating pickle files for user local model. i.e all per user reviews split into test and train

In [4]:
from util import *
user_id = ''
filename = 'pickle/'
user = load_pickle(filename+'userid.pickle')
ratings = load_pickle(filename+'rating.pickle')
business = load_pickle(filename+'business.pickle')
docs_csr = load_pickle(filename+'docs_csr.pickle')
unique_users = set(user)
unique_business = set(business)

with open("data/test_UID.dat", "r") as fh:
    lines = fh.readlines()


user2 = []
ratings2 = []
business2 = []
#docs_csr2 = []
for i in lines:
    user2.append(user[int(i)])
    ratings2.append(ratings[int(i)])
    business2.append(business[int(i)])
lines1 = map(int, lines)
filename = 'pickle/test/uid/'
save_pickle(user2, filename+'userid.pickle')
save_pickle(ratings2, filename+'rating.pickle')
save_pickle(business2, filename+'business.pickle')
save_pickle(docs_csr[lines1], filename+'docs_csr.pickle')

In [5]:
with open("data/train_UID.dat", "r") as fh:
    lines = fh.readlines()

user2 = []
ratings2 = []
business2 = []
#docs_csr2 = []
for i in lines:
    user2.append(user[int(i)])
    ratings2.append(ratings[int(i)])
    business2.append(business[int(i)])
lines1 = map(int, lines)
filename = 'pickle/train/uid/'
save_pickle(user2, filename+'userid.pickle')
save_pickle(ratings2, filename+'rating.pickle')
save_pickle(business2, filename+'business.pickle')
save_pickle(docs_csr[lines1], filename+'docs_csr.pickle')

### Creating Data for Business Local Model

In [6]:
filename = 'pickle/train/uid/'
user = load_pickle(filename+'userid.pickle')
ratings = load_pickle(filename+'rating.pickle')
business = load_pickle(filename+'business.pickle')
docs_csr = load_pickle(filename+'docs_csr.pickle')
unique_users = set(user)
unique_business = set(business)


datframeb={}
datframeb['BusinessId']=business
datframeb = pd.DataFrame(datframeb)
test_train_split_per_groupby(datframeb,'BusinessId',20)


In [7]:
with open("data/test_BusinessId.dat", "r") as fh:
    lines = fh.readlines()

user2 = []
ratings2 = []
business2 = []
#docs_csr2 = []
for i in lines:
    user2.append(user[int(i)])
    ratings2.append(ratings[int(i)])
    business2.append(business[int(i)])
lines1 = map(int, lines)
filename = 'pickle/test/businessid/'
save_pickle(user2, filename+'userid.pickle')
save_pickle(ratings2, filename+'rating.pickle')
save_pickle(business2, filename+'business.pickle')
save_pickle(docs_csr[lines1], filename+'docs_csr.pickle')

In [9]:
with open("data/train_BusinessId.dat", "r") as fh:
    lines = fh.readlines()

user2 = []
ratings2 = []
business2 = []
#docs_csr2 = []
for i in lines:
    user2.append(user[int(i)])
    ratings2.append(ratings[int(i)])
    business2.append(business[int(i)])
lines1 = map(int, lines)
filename = 'pickle/train/businessid/'
save_pickle(user2, filename+'userid.pickle')
save_pickle(ratings2, filename+'rating.pickle')
save_pickle(business2, filename+'business.pickle')
save_pickle(docs_csr[lines1], filename+'docs_csr.pickle')